In [ ]:
!pip install simple_image_download

In [10]:
from simple_image_download import simple_image_download as simp

my_downloader = simp.simple_image_download()
my_downloader.directory = '/content/drive/MyDrive/Colab Notebooks/deepMar/dataset/'
my_downloader.download('gun', limit=200)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/deepMar/
!ls

In [ ]:
!pip install --upgrade sympy

#Training and Testing Code

In [4]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_dir = "/content/drive/MyDrive/Colab Notebooks/deepMar/simple_images"
batch_size = 16
num_epochs = 5
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)
class_names = dataset.classes
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# DeepMAR Network, a commong feature extractor followed by simple-FCN layer. You can change both layers.
class ImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ImageClassifier, self).__init__()
        self.feature_extractor = models.resnet18(pretrained=True)  # resnet18 = feature extractor
        self.feature_extractor.fc = nn.Linear(self.feature_extractor.fc.in_features, num_classes)

    def forward(self, x):
        return self.feature_extractor(x)

model = ImageClassifier(num_classes=len(class_names)).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
def train(model, loader):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

# Testing Loop
def test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train and evaluate
for epoch in range(num_epochs):
    train_loss = train(model, train_loader)
    test_accuracy = test(model, test_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

 42%|████▏     | 8/19 [00:28<00:36,  3.30s/it]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 5/5 [00:07<00:00,  1.56s/it]


Epoch [1/5], Loss: 0.5438, Test Accuracy: 79.45%


100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


Epoch [2/5], Loss: 0.1980, Test Accuracy: 93.15%


100%|██████████| 5/5 [00:06<00:00,  1.38s/it]


Epoch [3/5], Loss: 0.0705, Test Accuracy: 95.89%


100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


Epoch [4/5], Loss: 0.0818, Test Accuracy: 86.30%


100%|██████████| 5/5 [00:06<00:00,  1.30s/it]

Epoch [5/5], Loss: 0.0937, Test Accuracy: 98.63%


# Inference

In [8]:
from PIL import Image

# Inference
def predict(model, image_path):
    model.eval()
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]

# Example inference
image_path = "/content/drive/MyDrive/Colab Notebooks/deepMar/simple_images/hockeystick/hockeystick_62.jpg"
predicted_class = predict(model, image_path)
print(f"Predicted class: {predicted_class}")

Predicted class: hockeystick


#Save model - Load model

In [ ]:
# Specify the path to save the model
model_save_path = "/content/drive/MyDrive/Colab Notebooks/deepMar/my_model.pth"

# Save the model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Load the model
model = ImageClassifier(num_classes=len(class_names)).to(device)
model.load_state_dict(torch.load(model_save_path))
model.eval()  # Set to evaluation mode for inference
print("Model loaded successfully")